In [1]:
from scipy.optimize import minimize
from skimage.color import deltaE_ciede2000, deltaE_cie76, rgb2lab
import numpy as np
import math
from collections import namedtuple


In [2]:
def clamp(low, x, high):
  return max(low, min(x, high))

In [3]:
RGB = namedtuple('RGB', ['r','g','b'])
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return RGB(*(int(hex_color[i:i+2], 16) for i in (0, 2, 4)))

def rgb_to_hex(rgb: RGB):
    return '#{:02x}{:02x}{:02x}'.format(*(map(lambda x: int(round(x)),rgb)))

In [4]:
def shade_func(color, offset):
    r,g,b = color
    offset *= 255
    return RGB(r - offset, g - offset, b - offset)

def tint_func(color, offset):
    r,g,b = color
    offset *= 255
    return RGB(r + offset, g + offset, b + offset)

ToneComponents=namedtuple('ToneComponents', ['ratio', 'value']) 
def tone_func(color, offset):
    r, g, b = color
    gr = offset.ratio * 255
    gray = RGB(gr, gr, gr)
    return (
        r * (1 - offset.value) + gray.r * offset.value,
        g * (1 - offset.value) + gray.g * offset.value,
        b * (1 - offset.value) + gray.b * offset.value
    )

In [5]:
class ColorNameFinder:
  def __init__(self, colors, deltaE=None):
    if deltaE is None:
      distance = deltaE_ciede2000
    self.distance = distance

    self.colors = [hex_to_rgb(color) for color in colors]

  def __distance(self, rgb1, rgb2):
    return self.deltaE(rgb2lab(rgb1), rgb2lab(rgb2)).mean()

  def __factory_objective(self, target, preprocessor=lambda x: x):
    def fn(x):
      x = preprocessor(x)
      color = self.colors[x[0]]
      offset, ratio = x[1], x[2]
      bound_offset = abs(offset)
      offsets = [
        shade_func(color, bound_offset), 
        tint_func(color, bound_offset), 
        tone_func(color, ToneComponents(ratio=ratio, value=offset))]
      least_error = min([(right, self.distance(target, right)) \
        for right in offsets], key = lambda x: x[1])[1]   
      return least_error

    return fn
  
  def __resolve_offset_type(self, sample, target, offset, ratio):
    bound_offset = abs(offset) 

    shade = shade_func(sample, bound_offset)
    tint = tint_func(sample, bound_offset)
    tone = tone_func(sample, ToneComponents(ratio=ratio, value=offset))

    lookup = {}
    lookup[shade] =  "shade"
    lookup[tint] =  "tint"
    lookup[tone] =  "tone"

    offsets = [shade, tint, tone]
    least_error = min([(right, self.distance(target, right)) for right in offsets], key = lambda x: x[1])[0]   

    return lookup[least_error]

  def nearest_color(self, target):
    target = hex_to_rgb(target)

    preprocessor=lambda x: (int(x[0]), x[1], x[2])
    objective = self.__factory_objective(target, preprocessor=preprocessor)
    search = [minimize( objective, 
                        (i, 0, 0),
                        bounds=[(i, i), (-1, 1), (0, 1)],
                        method='Powell') \
              for i, color in enumerate(self.colors)]
    best = min(search, key=lambda x: x.fun)
    indices = [i for i, v in enumerate(search) if v.fun == best.fun]
    index = min(indices, \
      key=lambda j: search[j].x[1])
    result = search[index]

    color_index = int(result.x[0])
    nearest_color = self.colors[color_index]
    _, offset, ratio = preprocessor(result.x)
  
    offset_type = self.__resolve_offset_type(nearest_color, target, offset, ratio)

    report = {
      "color": rgb_to_hex(nearest_color),
      "offset": {
        "type": offset_type,
        "value": offset if offset_type == 'tone' else abs(offset)
      }
    }
    if offset_type == 'tone':
      report["offset"]["white ratio"] = ratio

    return report


In [6]:
colors = ['#E0B0FF', '#FF0000', '#000000', '#0000FF']
target = '#DFAEFE'
agent = ColorNameFinder(colors)
agent.nearest_color(target)

{'color': '#e0b0ff',
 'offset': {'type': 'shade', 'value': 0.004674511436016945}}

In [9]:
colors = ['#1199ff', '#777775', '#acaaaa', '#0000FF']
target = '#999999'
agent = ColorNameFinder(colors)
agent.nearest_color(target)

{'color': '#acaaaa', 'offset': {'type': 'shade', 'value': 0.07370528284098173}}

In [8]:
colors = ['#1199ff', '#777777', '#aaaaaa', '#0000FF']
target = '#999999'
agent = ColorNameFinder(colors)
assert agent.nearest_color(target)['color'] == '#aaaaaa'

# this one fails because it fails to find the optimal for #aaaaaa
# see agent.distance(hex_to_rgb(target), shade_func(hex_to_rgb(colors[2]), 0.1333333333333333/2))

AssertionError: 

In [10]:
#agent.distance(hex_to_rgb(target), shade_func(hex_to_rgb(colors[0]), 0.9999999999998693))
agent.distance(hex_to_rgb(target), shade_func(hex_to_rgb(colors[2]), 0.1333333333333333/2))
#agent.distance(hex_to_rgb(target), tone_func(hex_to_rgb(colors[0]), ToneComponents(6.61069608225157e-05, 1)))

0.25097096868458113

In [11]:
#rgb_to_hex(shade_func(hex_to_rgb(colors[0]), 0.9999999999998693))
rgb_to_hex(shade_func(hex_to_rgb(colors[2]), 0.07370528284098173))

# tone_func(hex_to_rgb(colors[0]), ToneComponents(6.61069608225157e-05, 1))

'#999797'